In [376]:
# Imports
import pandas as pd
import time 
import math

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [377]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'uci-data/'
MODELS_PATH = DATA_PATH + 'models/raw-models/'
FEATURES = ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]
SAMPLE_SIZE = 50000
BATCH_SIZE = 32

In [378]:
# Load the data
train_set = pd.read_csv(DATA_PATH + "self-calculated/raw-train-data.txt", sep='\s+', header=None)
test_set = pd.read_csv(DATA_PATH + "self-calculated/raw-test-data.txt", sep='\s+', header=None)

train_set.columns=FEATURES
test_set.columns=FEATURES

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/var/folders/lt/w0169b7x5ml3psz3nly9vj3m0000gn/T/ipykernel_35356/2301537277.py:2: SyntaxWarning: invalid escape sequence '\s'
  train_set = pd.read_csv(DATA_PATH + "self-calculated/raw-train-data.txt", sep='\s+', header=None)
/var/folders/lt/w0169b7x5ml3psz3nly9vj3m0000gn/T/ipykernel_35356/2301537277.py:3: SyntaxWarning: invalid escape sequence '\s'
  test_set = pd.read_csv(DATA_PATH + "self-calculated/raw-test-data.txt", sep='\s+', header=None)


In [379]:
# Creating model
device = None

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class RawDataModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(FEATURES) - 1, 128)  # Adjust input size based on output of Conv1d
        self.relu1 = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.05)

        self.fc2 = nn.Linear(128, 64)  # Adjust input size based on output of Conv1d
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.05)

        self.fc3 = nn.Linear(64, 32)  # Adjust input size based on output of Conv1d
        self.relu3 = nn.ReLU()
        self.batchnorm3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.05)
        
        self.fc4 = nn.Linear(32, 16)
        self.silu = nn.SiLU()
        self.fc5 = nn.Linear(16, 12)
        self.softmax = nn.Softmax(dim=1)  # Use dim=1 for multi-class classification

    def forward(self, x):
        x = self.fc1(x)           # Shape: [batch_size, 512]
        x = self.relu1(x)         # Apply ReLU activation
        x = self.batchnorm1(x)
        x = self.dropout1(x)       # Apply dropout

        x = self.fc2(x)           # Shape: [batch_size, 512]
        x = self.relu2(x)         # Apply ReLU activation
        x = self.batchnorm2(x)
        x = self.dropout2(x)       # Apply dropout

        x = self.fc3(x)           # Shape: [batch_size, 512]
        x = self.relu3(x)         # Apply ReLU activation
        x = self.batchnorm3(x)
        x = self.dropout3(x)       # Apply dropout

        x = self.fc4(x)           # Shape: [batch_size, 32]
        x = self.silu(x)          # Apply SiLU activation
        x = self.fc5(x)           # Shape: [batch_size, 8]
        x = self.softmax(x)       # Apply Softmax for classification
        
        return x
    
model = RawDataModel().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [380]:
# Training and testing function
def train(dataloader, model, loss_fn, optimizer):
    # Get batch num
    num_batches = len(dataloader.dataset) / BATCH_SIZE
    i = 0

    # Set the model to train mode
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        i += 1
        if batch % 50 == 0:
            print(f"loss: {loss.item()}, batch: {i} out of {math.ceil(num_batches)}")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [381]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)[:SAMPLE_SIZE]
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)[:SAMPLE_SIZE]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [382]:
epochs = 5
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 2.4058189392089844, batch: 1 out of 1563
loss: 2.477020025253296, batch: 51 out of 1563
loss: 2.462775707244873, batch: 101 out of 1563
loss: 2.451233148574829, batch: 151 out of 1563
loss: 2.4184324741363525, batch: 201 out of 1563
loss: 2.333085536956787, batch: 251 out of 1563
loss: 2.146029472351074, batch: 301 out of 1563
loss: 2.4051644802093506, batch: 351 out of 1563
loss: 2.418623447418213, batch: 401 out of 1563
loss: 2.3104629516601562, batch: 451 out of 1563
loss: 2.2414867877960205, batch: 501 out of 1563
loss: 2.318122148513794, batch: 551 out of 1563
loss: 2.2933382987976074, batch: 601 out of 1563
loss: 2.273132801055908, batch: 651 out of 1563
loss: 2.21748948097229, batch: 701 out of 1563
loss: 2.1256773471832275, batch: 751 out of 1563
loss: 2.163008451461792, batch: 801 out of 1563
loss: 2.245713710784912, batch: 851 out of 1563
loss: 2.100432872772217, batch: 901 out of 1563
loss: 2.2738256454467773, batch: 951 out of 1

In [383]:
model_name = "cnn_8"
if True:
    torch.save(model.state_dict(), MODELS_PATH + model_name + ".pth")

    with open(MODELS_PATH + model_name + ".txt", "w") as f:
        f.write("Epochs: {}\n".format(epochs))
        f.write("Feature Set: {}\n".format(FEATURES))
        f.write("Model: {}\n".format(str(model)))
        f.write("Loss Function: {}\n".format("Cross Entropy Loss"))
        f.write("Optimizer: {}\n\n\n\n".format(str(optimizer)))
        f.write("Results: {}\n".format(perf_acc))
        f.write("Timer: {}s\n".format(round(perf_timer, 2)))

Saved PyTorch Model State to model.pth
